In [ ]:
## Création du fichier de départ pour la liste des pdf téléchargés

import os
import gzip
from lxml import etree

# Dossiers
base_dir = "/data/Notices_Xml-tei_de_Hal/"
output_gz = "/data/log/pdfs_deja_telecharges.txt.gz"

namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}
pdf_urls = set()
fichiers_xml = []

# Balayage des fichiers XML existants
for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            fichiers_xml.append(os.path.join(root, file))

print(f"{len(fichiers_xml)} fichiers XML trouvés")

# Extraction des URLs de PDF
for filepath in fichiers_xml:
    try:
        with open(filepath, 'rb') as f:
            xml = etree.parse(f)

        refs = xml.xpath('.//tei:edition[@type="current"]/tei:ref', namespaces=namespaces)

        for ref in refs:
            target = ref.get("target")
            ref_type = ref.get("type")
            if target and ref_type == "file" and target.lower().endswith(".pdf"):
                pdf_urls.add(target)

    except Exception as e:
        print(f"Erreur lecture {filepath} : {e}")

print(f"{len(pdf_urls)} URLs PDF extraites")

# Écriture dans le fichier gzip
with gzip.open(output_gz, 'wt', encoding='utf-8') as gz_file:
    for url in sorted(pdf_urls):
        gz_file.write(url + "\n")

print(f"Fichier {output_gz} écrit.")
